# 전처리 코드

## 라이브러리 불러오기

In [1]:
from typing import Tuple, List
import polars as pl
from pprint import pprint
import sys
from pathlib import Path
import psutil

# 상대 경로 사용
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / 'data'

# 맨 앞에 추가
if str(PROJECT_ROOT) in sys.path:
    sys.path.remove(str(PROJECT_ROOT))
sys.path.insert(0, str(PROJECT_ROOT))

# Python 내장 code 모듈 캐시만 임시 제거
if 'code' in sys.modules:
    del sys.modules['code']

# 이제 import
from code.loading import DataLoader

In [2]:
# 시스템의 논리적 CPU 코어 수를 기반으로 워커 수 결정
optimal_workers = psutil.cpu_count(logical=True)

print(f"사용할 워커 수: {optimal_workers}")

# 시스템의 전체 메모리 정보를 가져옴
mem = psutil.virtual_memory()

print(f"총 메모리: {mem.total / (1024**3):.2f} GB")
print(f"사용 가능 메모리: {mem.available / (1024**3):.2f} GB")
print(f"사용 중인 메모리: {mem.used / (1024**3):.2f} GB")
print(f"메모리 사용률: {mem.percent}%")

사용할 워커 수: 64
총 메모리: 125.14 GB
사용 가능 메모리: 111.84 GB
사용 중인 메모리: 13.30 GB
메모리 사용률: 10.6%


## 원본 데이터 불러오기

In [ ]:

loader2 = DataLoader(
    start=2020,
    end=2025,
    output_file='../output.parquet',
    max_workers=4
)

adapter = 'polars'
polars_kwargs = {
    'use_statistics': True,
    'parallel': 'auto',
    'low_memory': False,
    'rechunk': False,
    'cache': True,
}
df = loader2.load(adapter=adapter, **polars_kwargs)
df


📖 ../output.parquet 로딩 중... (adapter=polars)


In [5]:
# 전역 변수
BASE_COLS = [
    'mdr_report_key', 'adverse_event_flag', 'product_problems',
    'product_problem_flag', 'date_of_event', 'date_report', 
    'date_received', 'device_date_of_manufacturer', 'event_type',
    'previous_use_code', 'single_use_flag', 'report_source_code',
    'reprocessed_and_reused_flag', 'date_facility_aware', 'report_date',
    'report_to_fda', 'date_report_to_fda', 'report_to_manufacturer',
    'date_report_to_manufacturer', 'event_location', 
    'date_manufacturer_received', 'manufacturer_link_flag',
    'date_added', 'date_changed', 'pma_pmn_number',
    'suppl_dates_fda_received', 'suppl_dates_mfr_received'
]

# device_* 특정 컬럼들
DEVICE_PATTERNS = [
    '_brand_name', '_udi_di', '_device_report_product_code', 
    '_model_number', '_expiration_date_of_device', '_device_age_text',
    '_device_operator', '_implant_flag', '_manufacturer_d_name', 
    '_openfda_device_class', '_openfda_device_name', '_generic_name'
]

# patient_* 특정 컬럼들
PATIENT_PATTERNS = [
    '_patient_sequence_number', '_patient_age', '_patient_sex',
    '_patient_weight', '_patient_race', '_patient_problems',
    '_sequence_number_outcome'
]

# mdr_text_* 특정 컬럼들
MDR_TEXT_PATTERNS = ['_text', '_text_type_code']

## 사용할 컬럼만 1차 추출

In [6]:
def get_use_cols(
    df: pl.LazyFrame,
    base_cols: List[str] = BASE_COLS,
    device_patterns: List[str] = DEVICE_PATTERNS,
    patient_patterns: List[str] = PATIENT_PATTERNS,
    mdr_text_patterns: List[str] = MDR_TEXT_PATTERNS
) -> Tuple[List[str], List[str], List[str], List[str]]:
    
    total_cols = df.collect_schema().names()
    device_cols = [col for col in total_cols if col.startswith('device_') and 
                any(pattern in col for pattern in device_patterns)]
    patient_cols = [col for col in total_cols if col.startswith('patient_') and 
                    any(pattern in col for pattern in patient_patterns)]
    mdr_text_cols = [col for col in total_cols if col.startswith('mdr_text_') and 
                    any(col.endswith(pattern) for pattern in mdr_text_patterns)]

    # 합치기
    analysis_cols = base_cols + device_cols + patient_cols + mdr_text_cols
    analysis_cols = sorted(list(set(analysis_cols)), reverse=True)

    print(f"총 컬럼: {len(analysis_cols)}개")
    print(f"  - 기본 컬럼: {len(base_cols)}개")
    print(f"  - device_*: {len(device_cols)}개")
    print(f"  - patient_*: {len(patient_cols)}개")
    print(f"  - mdr_text_*: {len(mdr_text_cols)}개")

    return device_cols, patient_cols, mdr_text_cols, analysis_cols

device_cols, patient_cols, mdr_text_cols, analysis_cols = get_use_cols(df)

총 컬럼: 835개
  - 기본 컬럼: 27개
  - device_*: 622개
  - patient_*: 26개
  - mdr_text_*: 160개


## 타입 확인

In [7]:
target_col = 'adverse_event_flag'
df.select(
    pl.col(target_col).n_unique().alias(f'unique_{target_col}')
).collect()

df.select(
    pl.col(target_col).unique().sort().head(100).alias(f'head_{target_col}'),
    pl.col(target_col).unique().sort().tail(100).alias(f'tail_{target_col}'),
).collect().to_pandas()

,head_adverse_event_flag,tail_adverse_event_flag
0,None,None
1,N,N
2,Y,Y


In [8]:
def look_over_col(df: pl.LazyFrame, col: str):
    nunique = df.select(
        pl.col(col).n_unique().alias(f'unique_{col}')
    ).collect().item()
    
    print(f'{col}의 고유 개수: {nunique}')
    
    unique_df = df.select(
        pl.col(col).unique().sort().head(100).alias(f'head_{col}'),
        pl.col(col).unique().sort().tail(100).alias(f'tail_{col}'),
    ).collect().to_pandas()
    
    display(unique_df)

In [9]:
for col in BASE_COLS:
    look_over_col(df, col)

mdr_report_key의 고유 개수: 13642636


,head_mdr_report_key,tail_mdr_report_key
0,10000000,9999900
1,10000001,9999901
2,10000002,9999902
3,10000003,9999903
4,10000004,9999904
...,...,...
95,10000095,9999995
96,10000096,9999996
97,10000097,9999997
98,10000098,9999998


adverse_event_flag의 고유 개수: 3


,head_adverse_event_flag,tail_adverse_event_flag
0,None,None
1,N,N
2,Y,Y


product_problems의 고유 개수: 126967


...

,head_pma_pmn_number,tail_pma_pmn_number
0,None,SEEH.10
1,""" K181660""",SEEH10
2,""" K181699""",SEEH11
3,"""K031361 ""","SEP 3, 2020"
4,"""K042172 """,SERVO-S
...,...,...
95,09/25/2025,EXEMPT
96,09005811214,K113355
97,090909,K963413
98,09305,¿P100021


suppl_dates_fda_received의 고유 개수: 293029


,head_suppl_dates_fda_received,tail_suppl_dates_fda_received
0,None,12/31/2020;08/11/2021;01/13/2022;12/13/2023
1,*,12/31/2020;08/20/2021
2,*;02/22/2022,12/31/2020;08/25/2022
3,*;02/24/2020;09/10/2020;02/04/2022,12/31/2020;08/27/2021;12/02/2022;01/09/2023
4,*;02/26/2020,12/31/2020;08/28/2024
...,...,...
95,01/01/2024;06/13/2024,12/31/2024;08/07/2025
96,01/01/2024;06/20/2024,12/31/2024;10/10/2025
97,01/01/2024;07/03/2024,12/31/2024;10/15/2025
98,01/01/2024;08/02/2024,12/31/2024;12/31/2024


suppl_dates_mfr_received의 고유 개수: 286950


,head_suppl_dates_mfr_received,tail_suppl_dates_mfr_received
0,None,12/31/2021
1,*,12/31/2021;01/06/2022
2,*;*,12/31/2021;01/10/2022
3,*;*;*,12/31/2021;01/11/2022;01/14/2022;02/16/2022
4,*;*;*;*,12/31/2021;01/13/2022
...,...,...
95,*;*;07/09/2025;07/09/2025;07/09/2025,12/31/2024;08/08/2025
96,*;*;07/11/2024,12/31/2024;09/11/2025
97,*;*;07/12/2020,12/31/2024;12/31/2024
98,*;*;07/12/2023,12/31/2024;12/31/2024;12/31/2024


In [10]:
first_device_cols = [col for col in device_cols if col[7] == '0']

for col in first_device_cols:
    look_over_col(df, col)

device_0_brand_name의 고유 개수: 207688


,head_device_0_brand_name,tail_device_0_brand_name
0,None,\
1,!!! POWERFLEXX,\HEMOLUNG RAS
2,!!35X PROFLEXX,]
3,!!93-P PROFLEXX,`
4,!M1,`BYTE DAY ALIGNERS
...,...,...
95,(17-0144) ANK IMPL A11 D3.5 MM/L11 MM (IMPLANTS),"Ø8X 30MM BC IF SCRW, VENTED"
96,(17-0144) ANK IMPL A11 D3.5 MM/L11 MM(,Ø9MM TI ADOLESCENT LAT ENTRY FEMORAL NAIL-EX/3...
97,(17-0144) ANK IMPL A11 D3.5 MM/L11 MM(IMPLANTS),"Ø9X 20MM BC IF SCRW, VENTED"
98,(17-0146) ANK IMPL A14 D3.5 MM/L14 MM,"Ø9X 30MM BC IF SCRW, VENTED"


device_0_device_age_text의 고유 개수: 9161


,head_device_0_device_age_text,tail_device_0_device_age_text
0,None,9708 DA
1,0 DA,9709 DA
2,0 MO,971 DA
3,0 YR,9710 DA
4,1 DA,9711 DA
...,...,...
95,1068 DA,999 DA
96,1069 DA,DA
97,107 DA,MO
98,107 MO,NULL


device_0_device_operator의 고유 개수: 7


...

,head_device_0_openfda_device_class,tail_device_0_openfda_device_class
0,None,None
1,1,1
2,2,2
3,3,3
4,N,N
5,U,U
6,f,f


device_0_openfda_device_name의 고유 개수: 3218


,head_device_0_openfda_device_name,tail_device_0_openfda_device_name
0,None,"Unit, Heating, Powered"
1,"Tampon, Menstrual, Scented, Scented-Deodorized","Unit, Isolation, Surgical"
2,"11-Dehydro Thromboxane B2 Kit, Urinary","Unit, Liquid-Oxygen, Portable"
3,"Aberrometer, Ophthalmic","Unit, Neonatal Phototherapy"
4,"Ablation Catheter, Renal Denervation","Unit, Operative Dental"
...,...,...
95,"Alpha-1-Antitrypsin, Antigen, Antiserum, Control",Wrench
96,"Alpha-1-Lipoprotein, Antigen, Antiserum, Control",X-Ray Controls - Fluoroscopic
97,Alternate Controller Enabled Insulin Infusion ...,X-Ray Controls - Radiographic
98,"Amalgamator, Dental, Ac-Powered","X-Ray, Tomography, Computed, Dental"


device_0_udi_di의 고유 개수: 142321


,head_device_0_udi_di,tail_device_0_udi_di
0,None,M724TMA0410R1
1,(01)(10)NA,M724TMA04L1
2,(01)SW11677,M724TSCIIXL1
3,(01)SW12300,M724TSCIM1
4,(01)UNKNOWN,M72520201020
...,...,...
95,00(01)(21)5533,M986USSL22132600
96,00(01)(21)5664,MHC18020151
97,00(01)(21)5676,MHC18223313
98,00(01)(21)5686,N


In [11]:
first_patient_cols = [col for col in patient_cols if col[8] == '0']

for col in first_patient_cols:
    look_over_col(df, col)

patient_0_patient_age의 고유 개수: 1073


,head_patient_0_patient_age,tail_patient_0_patient_age
0,None,81
1,-1,81 DA
2,-1 DA,81 MO
3,-1 YR,81 WK
4,-11 DA,81 YR
...,...,...
95,0067 YR,N/A
96,01 DA,NA
97,01 YR,UKN
98,023 YR,UNK


patient_0_patient_problems의 고유 개수: 157736


,head_patient_0_patient_problems,tail_patient_0_patient_problems
0,None,"['Wound Dehiscence', 'Unspecified Infection', ..."
1,"['Abdominal Cramps', 'Cognitive Changes']","['Wound Dehiscence', 'Unspecified Infection', ..."
2,"['Abdominal Cramps', 'Constipation', 'Insuffic...","['Wound Dehiscence', 'Unspecified Infection', ..."
3,"['Abdominal Cramps', 'Cramp(s) /Muscle Spasm(s)']","['Wound Dehiscence', 'Unspecified Infection', ..."
4,"['Abdominal Cramps', 'Device Embedded In Tissu...","['Wound Dehiscence', 'Unspecified Infection', ..."
...,...,...
95,"['Abdominal Pain', 'Abscess', 'Adhesion(s)', '...","[None, 'Foreign Body In Patient']"
96,"['Abdominal Pain', 'Abscess', 'Adhesion(s)', '...","[None, 'Insufficient Information']"
97,"['Abdominal Pain', 'Abscess', 'Adhesion(s)', '...","[None, 'No Clinical Signs, Symptoms or Conditi..."
98,"['Abdominal Pain', 'Abscess', 'Adhesion(s)', '...","[None, 'Pneumoperitoneum']"


patient_0_patient_race의 고유 개수: 43


...

,head_patient_0_patient_sex,tail_patient_0_patient_sex
0,None,None
1,Female,Female
2,Male,Male
3,Prefer Not To Disclose,Prefer Not To Disclose
4,Unknown,Unknown


patient_0_patient_weight의 고유 개수: 991


,head_patient_0_patient_weight,tail_patient_0_patient_weight
0,None,75 KG
1,-1,750 KG
2,-1 KG,76
3,1,76 KG
4,1 KG,760
...,...,...
95,140,990 KG
96,140 KG,991 KG
97,141,994
98,141 KG,995 KG


patient_0_sequence_number_outcome의 고유 개수: 816


,head_patient_0_sequence_number_outcome,tail_patient_0_sequence_number_outcome
0,None,"['Required Intervention', ' L', ' H', ' D', ' O']"
1,"['Congenital Anomaly', ' D', ' H', ' L', ' O',...","['Required Intervention', ' L', ' H', ' D', ' ..."
2,"['Congenital Anomaly', ' D', ' H', ' L', ' R',...","['Required Intervention', ' L', ' H', ' D']"
3,"['Congenital Anomaly', ' D', ' H', ' L', ' R']","['Required Intervention', ' L', ' H', ' O', ' D']"
4,"['Congenital Anomaly', ' D', ' H', ' O']","['Required Intervention', ' L', ' H', ' O', ' S']"
...,...,...
95,"['Death', ' L', ' R', ' S', ' H', ' O']","['Required Intervention', ' S', ' O', ' L', ' H']"
96,"['Death', ' L', ' R', ' S']","['Required Intervention', ' S', ' O', ' L']"
97,"['Death', ' L', ' R']","['Required Intervention', ' S', ' O']"
98,"['Death', ' L', ' S', ' H']","['Required Intervention', ' S']"


In [12]:
first_mdr_cols = [col for col in mdr_text_cols if '_0_' in col]

for col in first_mdr_cols:
    look_over_col(df, col)

mdr_text_0_text의 고유 개수: 4228768


,head_mdr_text_0_text,tail_mdr_text_0_text
0,None,¿THE FAILURES LEADING TO MOTOR STALL (ERROR CO...
1,(B)(4).,¿THE FAILURES LEADING TO MOTOR STALL (ERROR CO...
2,(B)(4). CANCEL AND CLOSE. S/W VERSION 4.11J. ...,¿THE FAILURES LEADING TO MOTOR STALL (ERROR CO...
3,(B)(4). CURRENTLY IT IS UNKNOWN WHETHER OR NO...,¿THE FOLLOWING FIELDS HAVE BEEN UPDATED WITH C...
4,(B)(4). CURRENTLY IT IS UNKNOWN WHETHER OR NO...,¿THE FREE STYLE LIBRE 3 MEASURED MY HUSBANDS B...
...,...,...
95,"""...VENTILATOR WAS NOTED TO HAVE FLICKED INTO ...","ÖKÇESIZ, I., DÖNMEZ, H., VURAL, A., KARABIYIK,..."
96,"""032227-060322-002-C. (B)(4). CURRENTLY IT IS ...","ÖZDEMIR-VAN BRUNSCHOT D, HARRICH F, TEVS M, HO..."
97,"""04/18/24 EVALUATION TECH: TL EMN HP; CABLE, C...","ÖZDEMIR-VAN BRUNSCHOT D, HARRICH F, TEVS M, HO..."
98,"""1 PORT OPERATED ON 1 CHILD. STARTED BLEEDING ...","ï»¿Unique Complaint ID Number,Event Date,Date ..."


mdr_text_0_text_type_code의 고유 개수: 3


,head_mdr_text_0_text_type_code,tail_mdr_text_0_text_type_code
0,None,None
1,Additional Manufacturer Narrative,Additional Manufacturer Narrative
2,Description of Event or Problem,Description of Event or Problem


### 타입별 컬럼 모으기

In [ ]:
date_cols = ['date_of_event', 'date_report', 'date_manufacturer_received', 'date_received', 'date_added', 'suppl_dates_fda_received', 'suppl_dates_mfr_received', 'device_date_of_manufacturer', 'device_0_expiration_date_of_device', 'date_changed']

In [14]:
def summarize_date_gaps(
    lf: pl.LazyFrame,
    date_col1: str = None,
    date_col2: str = None,
    fmt: str = "%Y%m%d",
) -> pl.LazyFrame:
    return (
        lf
        .select([date_col1, date_col2])
        .drop_nulls()
        .with_columns([
            pl.col(date_col1)
              .cast(pl.Utf8)
              .str.strptime(pl.Date, format=fmt, strict=False)
              .alias(date_col1),
            pl.col(date_col2)
              .cast(pl.Utf8)
              .str.strptime(pl.Date, format=fmt, strict=False)
              .alias(date_col2)
        ])
        .with_columns([
            (pl.col(date_col1).cast(pl.Int64) - pl.col(date_col2).cast(pl.Int64))
                .alias("days_date1_minus_date2"),
        ])
        .select([
            pl.col("days_date1_minus_date2").min().alias("min_date1_date2"),
            pl.col("days_date1_minus_date2").max().alias("max_date1_date2"),
            pl.col("days_date1_minus_date2").median().alias("med_date1_date2"),
            pl.col("days_date1_minus_date2").mean().alias("mean_date1_date2"),
        ])
    )


In [15]:
df.select(date_cols).head(10).collect().to_pandas()

,date_of_event,date_report,date_manufacturer_received,date_received,date_added,suppl_dates_fda_received,suppl_dates_mfr_received,device_date_of_manufacturer,device_0_expiration_date_of_device,date_changed
0,20200708,20200708,20200708,20200720,20200720,09/14/2020,09/03/2020,None,20210923,20230718
1,20200707,20200707,20200707,20200720,20200720,08/24/2020,08/21/2020,None,None,20230718
2,20200504,20200720,20200702,20200720,20200720,None,None,20191029,None,20200729
3,20200710,20200710,20200710,20200720,20200720,None,None,20150223,None,20200724
4,20200622,20200630,20200629,20200720,20200720,None,None,20200109,20210430,20200803
5,20200313,20200720,None,20200720,20200720,None,None,20150315,20200315,20200727
6,20200513,20200720,None,20200720,20200720,None,None,None,None,20200727
7,20200309,20200720,None,20200720,20200720,None,None,20181211,20231211,20200908
8,20200628,20200720,20200628,20200720,20200720,None,None,20190601,None,20200831
9,20200404,20200720,None,20200720,20200720,None,None,20181031,20231031,20200727


In [16]:
df.select(['device_date_of_manufacturer','device_0_udi_di', 'device_5_udi_di']).filter(
    pl.col('device_date_of_manufacturer').is_not_null() & \
    pl.col('device_5_udi_di').is_not_null()
).collect().to_pandas()

,device_date_of_manufacturer,device_0_udi_di,device_5_udi_di


In [17]:
def look_over_col(df: pl.LazyFrame, col: str):
    nunique = df.select(
        pl.col(col).n_unique().alias(f'unique_{col}')
    ).collect().item()
    
    print(f'{col}의 고유 개수: {nunique}')
    
    unique_df = df.select(
        pl.col(col).unique().sort().head(100).alias(f'head_{col}'),
        pl.col(col).unique().sort().tail(100).alias(f'tail_{col}'),
    ).collect().to_pandas()
    
    display(unique_df)

In [18]:
df.select(analysis_cols).filter(
    pl.col('suppl_dates_mfr_received').is_not_null()
).head(10).collect().to_pandas()

,suppl_dates_mfr_received,suppl_dates_fda_received,single_use_flag,reprocessed_and_reused_flag,report_to_manufacturer,report_to_fda,report_source_code,report_date,product_problems,product_problem_flag,...,date_report_to_manufacturer,date_report_to_fda,date_report,date_received,date_of_event,date_manufacturer_received,date_facility_aware,date_changed,date_added,adverse_event_flag
0,09/03/2020,09/14/2020,None,N,None,*,Manufacturer report,None,"['Activation, Positioning or Separation Problem']",Y,...,None,None,20200708,20200720,20200708,20200708,None,20230718,20200720,N
1,08/21/2020,08/24/2020,None,N,None,*,Manufacturer report,None,['Loss of or Failure to Bond'],Y,...,None,None,20200707,20200720,20200707,20200707,None,20230718,20200720,Y
2,09/14/2020,09/21/2020,None,N,None,N,Manufacturer report,None,['Wireless Communication Problem'],Y,...,None,None,20200921,20200720,20200629,20200706,None,20230515,20200720,N
3,06/30/2020,08/11/2020,None,N,None,N,Manufacturer report,None,"['Unable to Obtain Readings', 'Failure of Devi...",Y,...,None,None,20200630,20200721,None,20200630,None,20230717,20200721,N
4,09/13/2020,09/21/2020,N,N,None,N,Manufacturer report,None,['No Device Output'],Y,...,None,None,20200921,20200721,20200625,20200626,None,20201010,20200721,N
5,07/22/2020,07/27/2020,N,N,None,*,Manufacturer report,None,['Adverse Event Without Identified Device or U...,N,...,None,None,20200726,20200721,20200707,20200707,None,20200729,20200721,Y
6,07/24/2020,08/21/2020,None,N,None,*,Manufacturer report,None,['Patient-Device Incompatibility'],N,...,None,None,20200821,20200722,20190101,20200622,None,20230517,20200722,Y
7,07/24/2020;11/13/2020,07/27/2020;11/18/2020,None,N,None,*,Manufacturer report,None,['Low Readings'],N,...,None,None,20201118,20200722,20200715,20200715,None,20230518,20200722,Y
8,09/01/2020,09/09/2020,N,N,None,Y,Manufacturer report,None,['Pumping Stopped'],Y,...,None,None,20200722,20200722,20200702,20200702,None,20200913,20200722,N
9,07/30/2020,08/11/2020,N,N,None,Y,Manufacturer report,None,"['Crack', 'Failure to Sense']",Y,...,None,None,20200722,20200722,20200630,20200630,None,20201023,20200722,N


In [19]:
date_col1 = 'date_added'
date_col2 = 'date_received'
date_gaps = summarize_date_gaps(df, date_col1, date_col2)
date_gaps.collect()

min_date1_date2,max_date1_date2,med_date1_date2,mean_date1_date2
i64,i64,f64,f64
-3647,1262,0.0,1.369733


In [20]:
df.select([date_col1, date_col2]).filter(
    pl.col(date_col1) < pl.col(date_col2)
).collect().to_pandas()

,date_added,date_received
0,20181018,20200710
1,20201220,20201221
2,20201212,20201213
3,20201220,20201221
4,20201220,20201221
5,20201220,20201221
6,20201220,20201221
7,20201220,20201221
8,20201220,20201221
9,20201220,20201221


In [21]:
def get_type_cols(
    df: pl.LazyFrame
):
    pass

## 제품군 확인

In [22]:
# 상표명, generic_name
cols = ['device_0_brand_name', 'device_0_generic_name', 'device_0_device_report_product_code', 'device_0_openfda_device_name']

df.select(pl.col(cols)).group_by('device_0_device_report_product_code').agg(
    pl.col('device_0_generic_name').n_unique().alias('unique_generic_name'),
    pl.col('device_0_openfda_device_name').n_unique().alias('unique_device_0_openfda_device_name'),
).collect().to_pandas().sort_values('unique_generic_name', ascending=False)

,device_0_device_report_product_code,unique_generic_name,unique_device_0_openfda_device_name
2762,JWH,2277,1
1718,DZE,1548,1
2044,LZO,1113,1
740,LXH,1091,1
1410,FPA,992,1
...,...,...,...
3206,EBO,1,1
3207,QKU,1,1
3242,QTI,1,1
3243,MYR,1,1


In [ ]:
# 상표명, 등록번호
cols = ['device_0_brand_name', 'device_0_generic_name']

# df.select(pl.col(cols)).group_by('device_0_device_report_product_code').agg(
#     pl.col('device_0_generic_name').n_unique().alias('unique_generic_name')
# ).collect().to_pandas().sort_values('unique_generic_name', ascending=False)

ColumnNotFoundError: unable to find column "device_0_device_report_product_code"; valid columns: ["device_0_brand_name", "device_0_generic_name"]

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'sink' <---
SELECT [col("device_0_brand_name"), col("device_0_generic_name")]
  Parquet SCAN [../output.parquet]
  PROJECT */2236 COLUMNS
  ESTIMATED ROWS: 13642636

In [ ]:
df.select(cols).filter(
    pl.col('device_0_device_report_product_code').str.contains('JWH')
).group_by('device_0_generic_name').agg(
    pl.col('device_0_generic_name').len().alias('cnt_generic_name'),
    pl.col('device_0_brand_name').mode().alias('mode_brand_name'),
    pl.col('device_0_openfda_device_name').mode().alias('mode_device_0_openfda_device_name'),
).collect().to_pandas().sort_values('cnt_generic_name', ascending=False)

## 결측 통일

In [ ]:
def analyze_null_values(df, analysis_cols, verbose=True):
    """
    컬럼별 결측치 분석 함수
    
    Parameters:
    -----------
    df : pl.LazyFrame
        분석할 LazyFrame
    analysis_cols : list
        분석할 컬럼 리스트
    verbose : bool, default=True
        결과를 출력할지 여부
    
    Returns:
    --------
    tuple : (null_stats, null_df)
        null_stats: 결측치 통계 딕셔너리 리스트
        null_df: 결측치 통계를 담은 DataFrame
    """
    if verbose:
        print("\n=== 결측치 분석 ===")
    
    null_stats = []
    total_rows = df.select(pl.len()).collect().item()
    all_cols = df.collect_schema().names()
    
    for col in analysis_cols:
        if col in all_cols:
            null_count = df.select(pl.col(col).null_count()).collect().item()
            null_pct = null_count / total_rows * 100
            null_stats.append({
                'column': col,
                'null_count': null_count,
                'null_pct': null_pct
            })
            if verbose:
                print(f"{col:45s}: {null_count:>10,}개 ({null_pct:>5.2f}%)")
        else:
            if verbose:
                print(f"{col:45s}: 컬럼 없음")
    
    null_df = pl.DataFrame(null_stats).sort('null_pct', descending=True)
    return null_stats, null_df


# 함수 사용
null_stats, null_df = analyze_null_values(df, analysis_cols)

In [ ]:
manu_cond = True
manu_cond &= pl.col('manufacturer_g1_name') != 'None'
manu_cond &= pl.col('manufacturer_g1_name') != 'UNK'

manu_filtered = df.filter(manu_cond)

columns = manu_filtered.collect_schema().names()

# 간단하게 streaming 모드만 활성화
cols_to_keep = []
batch_size = 100

for i in range(0, len(columns), batch_size):
    batch_cols = columns[i:i + batch_size]
    
    result = manu_filtered.select([
        pl.col(col).is_not_null().any().alias(col)
        for col in batch_cols
    ]).collect(engine='streaming')
    
    cols_to_keep.extend([
        col for col in batch_cols if result[col][0]
    ])

manu_filtered = manu_filtered.select(cols_to_keep)
total_rows = manu_filtered.select(pl.len()).collect()

In [ ]:
unk_lst = df.filter(pl.col('device_0_model_number').str.to_uppercase().str.contains(r'^UNK')).select('device_0_model_number').collect().to_series().to_list()
set(unk_lst)

In [ ]:
def filter_notna_rows(df: pl.LazyFrame, col) -> Tuple[pl.LazyFrame, pl.Expr, pl.DataFrame]:
    na_patterns = r'^None$|^UNK|^NOT APPLICABLE$|^N/A$|^NA$|^$|\s+$'
    notna_cond = True
    notna_cond &= ~pl.col(col).str.to_uppercase().str.contains(na_patterns)
    filtered = df.filter(notna_cond)
    notna_rows = filtered.select(pl.len()).collect()
    return filtered, notna_cond, notna_rows

In [ ]:
total_rows = df.select(pl.len()).collect().item()

udi_filtered, udi_notna_cond, udi_notna_rows = filter_notna_rows(df, 'device_0_udi_di')
model_filtered, model_notna_cond, model_notna_rows = filter_notna_rows(df, 'device_0_model_number')

coverage_analysis = df.select([
    udi_notna_cond.is_not_null().alias("has_udi"),
    model_notna_cond.is_not_null().alias("has_model")
]).group_by(["has_udi", "has_model"]).len().with_columns([
    pl.format("{}%", (pl.col("len").mul(100).truediv(total_rows).round(1)).alias("pct"))
])

# 실제 커버리지 계산
actual_coverage = df.select([
    (udi_notna_cond | model_notna_cond).mean().alias("either"),
    (udi_notna_cond & model_notna_cond).mean().alias("both")
])

coverage_analysis.collect(), actual_coverage.collect()

In [ ]:
df.filter(udi_notna_cond & ~model_notna_cond).first().select('device_0_udi_di').collect()

In [ ]:
device_cols = [f'device_{i}_udi_di' for i in range(50)]
model_cols = [f'device_{i}_model_number' for i in range(50)]

target_cols = device_cols
unique_values = set()

for c in target_cols:
    lst = df.select(pl.col(c).unique())
    unique_values.update(lst.collect().to_series().to_list())

print(len(unique_values))

In [ ]:
df.filter(pl.col('device_0_brand_name').str.to_uppercase().str.starts_with('ACCELERATOR')).collect().to_pandas()

In [ ]:
df.select(pl.col('patient_0_patient_sequence_number', 'patient_1_patient_sequence_number', 'patient_2_patient_sequence_number', 'patient_3_patient_sequence_number').is_not_null().sum()).collect()

In [ ]:
total_rows = df.select(pl.len()).collect()

manu_col = 'device_0_manufacturer_d_name'
udi_col = 'device_0_udi_di'
model_col = 'device_0_model_number'
manu_filtered, manu_notna_cond, manu_notna_rows = filter_notna_rows(df, manu_col)
udi_filtered, udi_notna_cond, udi_notna_rows = filter_notna_rows(df, udi_col)
model_filtered, model_notna_cond, model_notna_rows = filter_notna_rows(df, model_col)

print(manu_notna_rows / total_rows * 100)
print(udi_notna_rows / total_rows * 100)
print(model_notna_rows / total_rows * 100)

In [ ]:
total_rows = manu_filtered.select(pl.len()).collect()
manu_filtered.select('device_0_udi_di').null_count().collect() / total_rows

In [ ]:
columns = manu_filtered.collect_schema().names()
pprint(columns)

In [ ]:

mdr_cond = pl.col('mdr_text_79_text') != 'None'
device_cond = pl.col('device_47_device_report_product_code') != 'None'
patient_cond = pl.col('patient_3_patient_sequence_number') != 'None'

pprint(manu_filtered.filter(device_cond).collect().to_dicts())
# df.filter(pl.col('device_54_device_report_product_code') != 'None').collect()
# df.filter(pl.col('patient_3_patient_sequence_number') != 'None').collect()